# Object detection — MobileNet‑SSD (OpenCV)
This notebook detects objects in a single image using OpenCV DNN with the MobileNet‑SSD model.
- Model files are downloaded automatically on first run.
- Change `IMAGE_PATH` to use your own image.

In [ ]:
# 1) install / import dependencies
import sys, os, urllib.request, subprocess

def ensure_pkg(pkg_name, import_name=None):
    try:
        __import__(import_name or pkg_name)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])

ensure_pkg("opencv-python", "cv2")
ensure_pkg("matplotlib")
ensure_pkg("numpy")

import cv2
import numpy as np
import matplotlib.pyplot as plt
print("Dependencies imported.")

In [ ]:
# 2) download MobileNet-SSD model files if missing
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)
PROTO = os.path.join(MODEL_DIR, "MobileNetSSD_deploy.prototxt")
MODEL = os.path.join(MODEL_DIR, "MobileNetSSD_deploy.caffemodel")

PROTO_URL = "https://raw.githubusercontent.com/chuanqi305/MobileNet-SSD/master/MobileNetSSD_deploy.prototxt"
MODEL_URL = "https://github.com/chuanqi305/MobileNet-SSD/raw/master/MobileNetSSD_deploy.caffemodel"

if not os.path.exists(PROTO):
    print("Downloading prototxt...")
    urllib.request.urlretrieve(PROTO_URL, PROTO)
if not os.path.exists(MODEL):
    print("Downloading caffemodel (may take a moment)...")
    urllib.request.urlretrieve(MODEL_URL, MODEL)

# class labels for MobileNet-SSD (PASCAL VOC)
CLASSES = ["background","aeroplane","bicycle","bird","boat","bottle","bus","car","cat",
           "chair","cow","diningtable","dog","horse","motorbike","person","pottedplant",
           "sheep","sofa","train","tvmonitor"]

net = cv2.dnn.readNetFromCaffe(PROTO, MODEL)
print("Model loaded.")

In [ ]:
# 3) detection helper

def detect_objects(image, net, conf_threshold=0.4):
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)),
                                 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()
    out = image.copy()
    for i in range(0, detections.shape[2]):
        conf = float(detections[0, 0, i, 2])
        if conf < conf_threshold:
            continue
        idx = int(detections[0, 0, i, 1])
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        label = CLASSES[idx] if idx < len(CLASSES) else str(idx)
        color = (0, 255, 0)
        cv2.rectangle(out, (startX, startY), (endX, endY), color, 2)
        text = f"{label}: {conf:.2f}"
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(out, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return out

In [ ]:
# 4) load an image (edit IMAGE_PATH or upload) and run detection
IMAGE_PATH = "input.jpg"   # <- set your file path here

# fallback: download a sample if file not present
if not os.path.exists(IMAGE_PATH):
    sample_url = "https://upload.wikimedia.org/wikipedia/commons/7/7f/Golden_Retriever_puppy.jpg"
    print("No local image found; downloading sample image...")
    urllib.request.urlretrieve(sample_url, IMAGE_PATH)

img = cv2.imread(IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"Could not read {IMAGE_PATH}")

result = detect_objects(img, net, conf_threshold=0.4)

# display (convert BGR -> RGB for matplotlib)
plt.figure(figsize=(10,6))
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

# save output
out_path = "output_detected.jpg"
cv2.imwrite(out_path, result)
print("Result saved to", out_path)

## Notes & next steps
- Change `IMAGE_PATH` to run on your own image.  
- To process a folder or video, I can add a batch/video cell.  
- Want this notebook committed to your GitHub repo? Reply "commit" and I'll add & push it.